# Product/Face Detection/Recognition Service

This notebook, contain a novel approach for refinement of the data for YOLO, and trained YOLO model for face detection/recognition, that will be compitable with Product and logo detection and recognition.

### Install Dependencies and Setup

In [2]:
#!pip install ultralytics

Setup and Get Data

In [2]:
#!git clone https://github.com/aziz-ullah-khan/AdvanceFaceDetection.git

In [3]:
import os
import time
import uuid
import cv2

In [4]:
IMAGES_PATH = os.path.join('data','train','images')
number_images = 90

In [5]:
def main():
    windowName = "Main"
    cv2.namedWindow(windowName)
    cap = cv2.VideoCapture(0)

    print('Width :' + str(cap.get(3)))
    print('Height :' + str(cap.get(4)))

    cap.set(3, 620)
    cap.set(4, 720)

    for imgnum in range(number_images):
        print('Collecting image {}'.format(imgnum))
        ret, frame = cap.read()
        imgname = os.path.join(IMAGES_PATH,f'{str(uuid.uuid1())}.jpg')
        cv2.imshow('Main', frame)
        cv2.imwrite(imgname, frame)
        time.sleep(1.5)   

        if cv2.waitKey(1) & 0xFF == ord('q'):

            cv2.destroyAllWindow()
            cap.release()    

if __name__== "__main__":
    main() 

2024-10-31 17:47:37.047 python[5697:50014] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist.


Width :1280.0
Height :720.0


In [1]:
!labelme

Skipping unexpected key in config: ai
Skipping unexpected key in config: fill_drawing
Skipping unexpected key in config: ai_polygon
Skipping unexpected key in config: ai_mask
Skipping unexpected key in config: show_all_polygons
Skipping unexpected key in config: hide_all_polygons
Skipping unexpected key in config: toggle_all_polygons


## Importing required Packages

In [1]:
import os
import json
from ultralytics import YOLO

## Configuration

In [3]:
#cd ObjectDetection/

[Errno 2] No such file or directory: 'ObjectDetection/'
/Users/leonzlotnik/Desktop/GreenBus/ObjectDetection


In [8]:
pwd

'/Users/leonzlotnik/Desktop/GreenBus/ObjectDetection'

In [11]:
data_folder = './data/'
label_dir_train = './data/train/labels'
label_dir_val = './data/val/labels'
img_dir_train = './data/train/images'
img_dir_val = './data/val/images'
output_dir_train = label_dir_train  # Labels will be saved in the same folder
output_dir_val = label_dir_val

In [12]:
#Move Matching Labels

In [14]:
for folder in ['train','test','val']:
    for file in os.listdir(os.path.join('data', folder, 'images')):
        
        filename = file.split('.')[0]+'.json'
        existing_filepath = os.path.join('data','train','labels', filename)
        if os.path.exists(existing_filepath): 
            new_filepath = os.path.join('data',folder,'labels',filename)
            os.replace(existing_filepath, new_filepath) 

## Accessories

In [15]:
# to convert JSON labels to YOLO format
def convert_to_yolo_format(json_data, img_width, img_height):
    yolo_labels = []
    for shape in json_data['shapes']:
        if shape['label'] == 'shampoo':
            points = shape['points']

            x1, y1 = points[0]
            x2, y2 = points[1]


            x_center = (x1 + x2) / 2 / img_width
            y_center = (y1 + y2) / 2 / img_height
            width = (x2 - x1) / img_width
            height = (y2 - y1) / img_height

            # Append class_id (face = 0) and normalized coordinates
            yolo_labels.append(f"0 {x_center} {y_center} {width} {height}")
    return yolo_labels

# Convert all JSON files to YOLO format
for filename in os.listdir(data_folder):
    if filename.endswith('.json'):
        json_path = os.path.join(data_folder, filename)
        with open(json_path, 'r') as f:
            data = json.load(f)

        img_width = data['imageWidth']
        img_height = data['imageHeight']
        yolo_labels = convert_to_yolo_format(data, img_width, img_height)

        # Save YOLO labels to a .txt file with the same name as the image
        txt_filename = filename.replace('.json', '.txt')
        txt_path = os.path.join(data_folder, txt_filename)
        with open(txt_path, 'w') as f:
            f.write('\n'.join(yolo_labels))


In [16]:
def convert_to_yolo_format(json_file, img_width, img_height):
    """Converts JSON bounding boxes to YOLO format."""
    yolo_data = []
    with open(json_file, 'r') as f:
        data = json.load(f)
        shapes = data['shapes']

        for shape in shapes:
            label = 0  # Assuming 'face' is the only class, thus class_id is 0
            points = shape['points']
            x_min, y_min = points[0]
            x_max, y_max = points[1]

            # Compute YOLO format
            x_center = (x_min + x_max) / 2.0 / img_width
            y_center = (y_min + y_max) / 2.0 / img_height
            width = (x_max - x_min) / img_width
            height = (y_max - y_min) / img_height

            yolo_data.append(f"{label} {x_center} {y_center} {width} {height}")

    return yolo_data

def convert_json_to_txt(label_dir, output_dir, img_dir):
    """Converts all JSON label files in a directory to YOLO .txt format."""
    for json_filename in os.listdir(label_dir):
        if json_filename.endswith('.json'):
            json_path = os.path.join(label_dir, json_filename)
            with open(json_path, 'r') as f:
                data = json.load(f)
                image_path = os.path.join(img_dir, os.path.basename(data['imagePath']))
                img_width = data['imageWidth']
                img_height = data['imageHeight']

                yolo_data = convert_to_yolo_format(json_path, img_width, img_height)

                # Write YOLO label file
                output_txt_path = os.path.join(output_dir, json_filename.replace('.json', '.txt'))
                with open(output_txt_path, 'w') as txt_file:
                    txt_file.write("\n".join(yolo_data))

# Convert train and val sets
convert_json_to_txt(label_dir_train, output_dir_train, img_dir_train)
convert_json_to_txt(label_dir_val, output_dir_val, img_dir_val)


## Model Training

In [17]:
import os

data_config_path = os.path.abspath("./data/config.yaml")

model = YOLO('yolov8n.pt')

# Run training with resolved path
model.train(data=data_config_path, epochs=100, imgsz=640, batch=8)

New https://pypi.org/project/ultralytics/8.3.26 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.23 🚀 Python-3.12.4 torch-2.2.2 CPU (Intel Core(TM) i5-8279U 2.40GHz)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/Users/leonzlotnik/Desktop/GreenBus/ObjectDetection/data/config.yaml, epochs=100, time=None, patience=100, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_mas

train: Scanning /Users/leonzlotnik/Desktop/GreenBus/ObjectDetection/data/train/l

train: New cache created: /Users/leonzlotnik/Desktop/GreenBus/ObjectDetection/data/train/labels.cache


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


/Users/leonzlotnik/opt/anaconda3/lib/python3.12/site-packages/albumentations/__init__.py:13: UserWarning: A new version of Albumentations is available: 1.4.21 (you have 1.4.16). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()
val: Scanning /Users/leonzlotnik/Desktop/GreenBus/ObjectDetection/data/val/label

...

                   all         15         14      0.996          1      0.995      0.655



100 epochs completed in 0.775 hours.
Optimizer stripped from runs/detect/train2/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train2/weights/best.pt, 6.2MB

Validating runs/detect/train2/weights/best.pt...
Ultralytics 8.3.23 🚀 Python-3.12.4 torch-2.2.2 CPU (Intel Core(TM) i5-8279U 2.40GHz)
Model summary (fused): 168 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  m


                   all         15         14      0.996          1      0.995      0.707
Speed: 3.6ms preprocess, 76.7ms inference, 0.0ms loss, 1.2ms postprocess per image
Results saved to runs/detect/train2


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x160855730>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048,    

## Model Evaluation

In [18]:
# Validate the model
metrics = model.val()
print(metrics)


Ultralytics 8.3.23 🚀 Python-3.12.4 torch-2.2.2 CPU (Intel Core(TM) i5-8279U 2.40GHz)
Model summary (fused): 168 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /Users/leonzlotnik/Desktop/GreenBus/ObjectDetection/data/val/label
                 Class     Images  Instances      Box(P          R      mAP50  m


                   all         15         14      0.996          1      0.995      0.707
Speed: 4.2ms preprocess, 74.3ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to runs/detect/train22
ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x16036d940>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.0330

## Test on all images

In [19]:
test_images_folder = './data/test/images'

# Get a list of all image file paths in the directory
image_paths = [os.path.join(test_images_folder, file) for file in os.listdir(test_images_folder) if file.endswith(('jpg', 'jpeg', 'png'))]

# Loop through each image in the directory
for image_path in image_paths:
    # Apply the model on each image
    results = model(image_path)

    # Access the first result (assuming the model returns a list of results)
    # and display it using a library like PIL or Matplotlib
    if results:
        results[0].show()  # Assuming `show()` is a method in your results object
    else:
        print(f"No result for image {image_path}")


image 1/1 /Users/leonzlotnik/Desktop/GreenBus/ObjectDetection/data/test/images/8c3e8ec2-97e2-11ef-9ade-acde48001122.jpg: 480x640 1 shampoo, 137.2ms
Speed: 4.6ms preprocess, 137.2ms inference, 0.7ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 /Users/leonzlotnik/Desktop/GreenBus/ObjectDetection/data/test/images/8fe61d9c-97e2-11ef-9ade-acde48001122.jpg: 480x640 1 shampoo, 115.5ms
Speed: 2.1ms preprocess, 115.5ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 /Users/leonzlotnik/Desktop/GreenBus/ObjectDetection/data/test/images/8b54ac8a-97e2-11ef-9ade-acde48001122.jpg: 480x640 1 shampoo, 106.1ms
Speed: 3.6ms preprocess, 106.1ms inference, 1.1ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 /Users/leonzlotnik/Desktop/GreenBus/ObjectDetection/data/test/images/85d57d52-97e2-11ef-9ade-acde48001122.jpg: 480x640 1 shampoo, 114.1ms
Speed: 2.4ms preprocess, 114.1ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 /

## Save Model

In [21]:
# After training is completed
import torch
torch.save(model.state_dict(), "best.pt")


## Conclusion

With the comparison to all models, VGG, facenet, Multi-Task CNN, etc, YOLO model is performing well.